In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

from imgreg.models.radon import RadonSolver
from imgreg.util.helpers import image_save_back_tf, rot_tr_gen, solver_gen
from imgreg.util.io import DirectoryView

In [ ]:
image_path_src = input("Enter the path to the image data:\n")

image_path_dest = input("Enter the path for the image output:\n")

if not os.path.exists(image_path_dest):
    os.makedirs(image_path_dest)

file_pattern = input("Enter a file-pattern to filter the files (e.g. '*.jpg'):\n")

step = 10

In [ ]:
d_view = DirectoryView(image_path_src, file_pattern=file_pattern)
fnames = [file for i, file in enumerate(sorted(d_view.files)) if not i % step]

## Radon solver

In [ ]:
%%time
ref_img = np.array(Image.open(f"{image_path_src}/test00108.jpg"))
ras = RadonSolver(ref_img=ref_img)
ras.UPSAMPLING.value = 20

radg = solver_gen(d_view, ras, step)

rad_rot_tr_arr = np.array(list(rot_tr_gen(radg)))

plt.plot(rad_rot_tr_arr[:, -1])
plt.xlabel("# image")
plt.ylabel("NormRel_L2")
plt.show()

print(f"Overall NormRel_L2: {sum(rad_rot_tr_arr[:,-1])}")

### save values to file

In [ ]:
df_out = pd.DataFrame(
    rad_rot_tr_arr,
    index=fnames,
    columns=[
        "tr_x",
        "tr_y",
        "tr_err",
        "rot",
        "rot_err",
        "NormRel_L2",
    ],
)
df_out.to_csv(f"radon-{step}.csv")
df_out

### load values from file and offset for angle plot

In [ ]:
df_in = pd.read_csv(f"radon-{step}.csv", index_col=0, sep=",")
rad_rot_tr_arr = df_in.to_numpy()
fnames = df_in.index

In [ ]:
rad_rot_tr_arr[:, 3] -=15
plt.plot(rad_rot_tr_arr[:, 3])
plt.xlabel("# image")
plt.ylabel("angle")
plt.show()

### write out back transformed images

In [ ]:
image_save_back_tf(rad_rot_tr_arr, fnames, image_path_src, image_path_dest)